In [1]:
import torch
import numpy as np

## Custom Dataset

* torch.utils.data 의 Dataset을 상속받으며 다음을 구현해야함
  * __init__
  * __len__
  * __getitem__

In [55]:
from torch.utils.data import Dataset

X = np.random.rand(1000, 5) * 10.0
y = X.dot(np.array([1.0, 2.0, 3.0, 4.0, 5.0])) + 6.0 + np.random.rand(1000)

class CustomImageDataset(Dataset):

    def __init__(self, X, y, dtype=None):
        self.X = X
        self.y = y
        if dtype:
            self.X = torch.as_tensor(self.X, dtype=dtype)
            self.y = torch.as_tensor(self.y, dtype=dtype)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

## Dataloader

* torch.utils.data의 Dataloader의 기능은 다음과 같음
  * dataset를 받아 batch 단위로 iterate 시켜줌
  * 매 epoch마다 shuffle을 실행할 수 있음
  * 이 과정을 멀티쓰레딩으로 가속시킬 수 있음

In [58]:
from torch.utils.data import DataLoader

dataset = CustomImageDataset(X, y, dtype=torch.float32)
dataloader = DataLoader(dataset, batch_size=5, shuffle=True)
for step, (X_, y_) in enumerate(dataloader):
    print(X_, y_)
    if step == 3:
        break

tensor([[5.6075, 7.0339, 4.3125, 5.0870, 1.6022],
        [2.7735, 2.8427, 6.4811, 1.6801, 8.4714],
        [3.4545, 4.3071, 2.5694, 4.4552, 0.3265],
        [1.7226, 2.5115, 5.7192, 5.0990, 0.5497],
        [7.1981, 5.1063, 9.0864, 9.4210, 7.6188]]) tensor([ 67.5757,  83.2501,  45.7142,  53.8314, 126.8450])
tensor([[0.9450, 1.1431, 7.1945, 8.5053, 7.8193],
        [0.1176, 6.8820, 3.2056, 9.9035, 4.4179],
        [5.5034, 9.8880, 8.9753, 9.7322, 4.9221],
        [8.2694, 8.4153, 2.2376, 4.0464, 8.7692],
        [7.2636, 2.2066, 3.0165, 6.4372, 3.1057]]) tensor([104.7104,  91.6585, 122.3220,  98.7609,  68.7350])
tensor([[9.3129, 0.3530, 7.9980, 4.7830, 3.0471],
        [4.0546, 3.6643, 7.4553, 7.1421, 1.1355],
        [8.4149, 5.8702, 8.4062, 8.5694, 5.0408],
        [0.0157, 0.1927, 6.5550, 8.3439, 4.6678],
        [8.7862, 9.7474, 0.2295, 3.2254, 6.7230]]) tensor([ 74.9420,  74.3410, 111.5862,  83.0658,  82.3148])
tensor([[0.4326, 1.0500, 3.4490, 0.1473, 8.1127],
        [1.3455, 2.7

## Mini Batch Gradient Descent

In [59]:
import torch.nn as nn
from torch.optim import SGD

In [78]:
class LinearModel(nn.Module):
    
    def __init__(self):
        super(LinearModel, self).__init__()
        self.layer = nn.Linear(5,1)
    
    def forward(self, X):
        return self.layer(X)

In [91]:
model = LinearModel()

n_epochs = 100
loss_fn = nn.MSELoss()
optimizer = SGD(model.parameters(), lr=0.001)

for epoch in range(n_epochs):
    for step, (X_, y_) in enumerate(dataloader):
        
        y_pred = model(X_)
        loss = loss_fn(y_pred, y_.unsqueeze(1))
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    if (epoch+1)%10 == 0:
        W, b = list(model.parameters())
        print(f'epoch : {epoch+1},  loss : {loss:.3f}, W : {W.detach().numpy()[0]}, b : {b.item():.3f}')

epoch : 10,  loss : 1.100, W : [1.1693529 2.1699708 3.1719947 4.1801596 5.162608 ], b : 2.167
epoch : 20,  loss : 0.342, W : [1.1287837 2.124685  3.14051   4.146009  5.117589 ], b : 3.137
epoch : 30,  loss : 0.660, W : [1.1022599 2.1164095 3.117995  4.1045623 5.0952406], b : 3.888
epoch : 40,  loss : 0.228, W : [1.0843624 2.0729866 3.0743763 4.0753584 5.066502 ], b : 4.469
epoch : 50,  loss : 0.509, W : [1.0648937 2.0521774 3.0595744 4.053874  5.057447 ], b : 4.918
epoch : 60,  loss : 0.451, W : [1.0545204 2.0416481 3.0498443 4.049986  5.0416665], b : 5.268
epoch : 70,  loss : 0.153, W : [1.0447928 2.0369017 3.040929  4.0424566 5.035152 ], b : 5.539
epoch : 80,  loss : 0.206, W : [1.0315843 2.0278256 3.024155  4.026033  5.0252953], b : 5.747
epoch : 90,  loss : 0.144, W : [1.0207733 2.0228195 3.0219462 4.017365  5.0225353], b : 5.909
epoch : 100,  loss : 0.065, W : [1.0260134 2.0178342 3.0116835 4.012835  5.0092225], b : 6.034
